<a href="https://colab.research.google.com/github/VielF/ColabProjects/blob/main/Segmenta%C3%A7%C3%A3o_por_Crescimento_de_Regi%C3%B5es_Parte_2_Algoritmos_Avan%C3%A7ados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Universidade do Vale do Itajaí<br>
Escola Politécnica<br>
Processamento Digital de Imagens

# Exercício Segmentação Clássica - Sem Deep Learning

### Tutoriais da OpenCV

- https://docs.opencv.org/master/d9/df8/tutorial_root.html
- https://www.geeksforgeeks.org/opencv-python-tutorial/

# Segmentação por Crescimento de Regiões - Parte 2 - Algoritmos Avançados

A busca pelo sonho de consumo do algoritmo genérico para dividir uma imagem nos objetos ali representados levou ao desenvolvimento de algoritmos de segmentação bastante elaborados, que tentavam unir várias técnicas diferentes e várias representações diferentes do conteúdo de uma imagem: grafos dos objetos na imagem, representação de longos gradientes suaves por grandes regiões da imagem representando o mesmo objeto que variava de luminosidade, similaridade de pixel baseada em características específicas de imagens de satélite ou outras e por aí vai. Alguns players de peso como a NASA tentaram a sua sorte neste tipo de algoritmo.

A maioria desses algoritmos foram implementados em linguagens de programação compiladas e existiram apenas na forma de executáveis que podiam ser baixados ou de código fonte em linguagem C ou C++. Como a pesquisa nesses algoritmos é anterior ao grande sucesso da linguagem Python, que estamos vendo agora, e as pesquisas agora estão voltadas para o desenvolvimento de algoritmos baseados em aprendizado profundo, muito poucos dos algoritmos que discutimos na nossa aula de segmentação avançada acabaram recebendo uma implementação em Python. O algoritmo de segmentação baseado em gráficos de Felzenszwalb & Huttenlocher, que possui um nome tão prosaico que ele só é citado pelas suas iniciais FH, é um dos poucos exemplos que nós podemos executar em Python. Abaixo vão alguns exemplos de sua execução.



### Inicializações

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Métodos baseados em gráficos: o algoritmo Felzenszwalb & Huttenlocher



### SciKit F&H com imagens em tons de cinza

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, interactive, interact_manual
from skimage.data import astronaut
from skimage.segmentation import felzenszwalb
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
import cv2

image = cv2.imread("/content/dataSeg/ct-02.jpg", cv2.IMREAD_GRAYSCALE)
#image = img_as_float(img[::2, ::2])


def my_fh(scale=100, sigma=0.5, min_size=50, colormap='magma'):
    global image
    colormap = eval('plt.cm.' + colormap)
    segments_fh = felzenszwalb(image, scale=scale, sigma=sigma, min_size=min_size)

    fig, ax = plt.subplots(ncols=2, figsize=(15, 7), sharex=True, sharey=True)

    ax[0].imshow(mark_boundaries(image, segments_fh))
    ax[0].set_title('Original com limites: F&H')

    ax[1].imshow(segments_fh, cmap=colormap, interpolation='nearest')
    ax[1].set_title('Segmentos: Felzenszwalb & Huttenlocher')

    for a in ax:
        a.set_axis_off()


    plt.tight_layout()
    plt.show()

interactive(my_fh, scale=100, sigma=0.5, min_size=50, colormap = ['nipy_spectral', 'hot', 'magma', 'seismic'])

interactive(children=(IntSlider(value=100, description='scale', max=300, min=-100), FloatSlider(value=0.5, des…

### SciKit F&H with Color Images

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, interactive, interact_manual
from skimage.data import astronaut
from skimage.segmentation import felzenszwalb
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float

img = cv2.imread("/content/dataSeg/car-01.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

image = img_as_float(img[::2, ::2])


def my_fh(scale=100, sigma=0.5, min_size=50):
    global image
    segments_fh = felzenszwalb(image, scale=scale, sigma=sigma, min_size=min_size)

    fig, ax = plt.subplots(ncols=2, figsize=(19, 7), sharex=True, sharey=True)

    ax[0].imshow(mark_boundaries(image, segments_fh))
    ax[0].set_title('Original com limites: F&H')

    ax[1].imshow(segments_fh, cmap=plt.cm.nipy_spectral, interpolation='nearest')
    ax[1].set_title('Segmentos: Felzenszwalb & Huttenlocher')

    for a in ax:
        a.set_axis_off()


    plt.tight_layout()
    plt.show()

interactive(my_fh, scale=100, sigma=0.5, min_size=50)

interactive(children=(IntSlider(value=100, description='scale', max=300, min=-100), FloatSlider(value=0.5, des…

### Implementação PEGBIS F&H

Aqui usaremos o **PEGBIS** *(Python Efficient Graph-Based Image Segmentation)*, uma implementação Python por Ghassem Alaee do "Efficient Graph-Based Image Segmentation" paper written by P. Felzenszwalb, D. Huttenlocher. O documento está disponível: http://cs.brown.edu/~pff/papers/seg-ijcv.pdf. A implementação C++ foi escrita pelo autor e está disponível em: http://cs.brown.edu/~pff/segment/.  Você encontrará a implementação do autor original em https://github.com/salaee/pegbis. A versão que estamos usando aqui teve algumas adaptações e atualizações do Python do Prof. Aldo von Wangenheim.

In [ ]:
# Faça isso uma vez (ou sempre que reiniciar o kernel se estiver usando o Colab)
# O código abaixo baixa a versão armazenada no github
# parâmetros wget:
# --backups=1 : renomeia o arquivo original com o sufixo .1 e grava o novo arquivo no nome de arquivo pretendido
# -q : execute silenciosamente, a menos que haja um erro
!wget --backups=1 -q https://raw.githubusercontent.com/awangenh/pegbis/master/segment_graph.py
!wget --backups=1 -q https://raw.githubusercontent.com/awangenh/pegbis/master/filter.py
!wget --backups=1 -q https://raw.githubusercontent.com/awangenh/pegbis/master/fh_segment.py
!wget --backups=1 -q https://raw.githubusercontent.com/awangenh/pegbis/master/disjoint_set.py

In [ ]:
# --------------------------------------------------------------------------------
# Segment an image:
# Returns a color image representing the segmentation.
#
# Inputs:
#           in_image: image to segment.
#           sigma: to smooth the image.
#           k: constant for threshold function.
#           min_size: minimum component size (enforced by post-processing stage).
#
# Returns:
#           num_ccs: number of connected components in the segmentation.
# --------------------------------------------------------------------------------
from scipy import ndimage
import matplotlib.pyplot as plt
from filter import *
from segment_graph import *
from fh_segment import *
import time
from ipywidgets import interact, interactive, interact_manual

sigma = 0.5
scale = 100
min_size = 50
input_path = "/content/dataSeg/car-01.jpg"

# Carregando a imagem
#input_image = ndimage.imread(input_path, flatten=False, mode=None)
input_image = plt.imread(input_path)

def my_pegbis(scale=100, sigma=0.5, min_size=50):
    global input_image

    print("processing...")
    output_image, elapsed_time = fh_segment(input_image, sigma, scale, min_size)
    print("Execution time: " + str(int(elapsed_time / 60)) + " minute(s) and " + str(int(elapsed_time % 60)) + " seconds")
    # exibindo o resultado
    fig, ax = plt.subplots(ncols=2, figsize=(14, 7), sharex=True, sharey=True)

    ax[0].imshow(input_image)
    ax[0].set_title('Original')

    ax[1].imshow(output_image)
    ax[1].set_title('Segments: Felzenszwalb & Huttenlocher')

    for a in ax:
        a.set_axis_off()


    plt.tight_layout()
    plt.show()

interact_manual(my_pegbis, scale=(1, 1000), sigma=(0.1, 3.0), min_size=(10, 1000))

interactive(children=(IntSlider(value=100, description='scale', max=1000, min=1), FloatSlider(value=0.5, descr…

<function __main__.my_pegbis(scale=100, sigma=0.5, min_size=50)>

# Créditos

* Truques gerais para exibir imagens vieram daqui: https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_image_display/py_image_display.html
* Também usamos algumas dicas gerais de: https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_segmentations.html#sphx-glr-auto-examples-segmentation-plot-segmentations-py
* Também usamos material do Professor Aldo von Wangenheim da UFSC